In [1]:
import os
cache_dir = "C:/huggingface_cache"
os.environ['HF_HOME'] = cache_dir
os.environ['TRANSFORMERS_CACHE'] = cache_dir # 만약을 위해 이중으로 설정

In [2]:
# 📚 필수 라이브러리 설치 및 임포트
import numpy as np
import pandas as pd
import os
import pickle
import joblib

# Kaggle 입력 데이터 확인
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import re
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"GPU 사용 가능: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")



GPU 사용 가능: True
사용 디바이스: cuda


In [3]:
# ==========================================================
# 📊 데이터 로드 및 기본 정보 확인
train_df = pd.read_csv('train.csv')

print(f"🔍 데이터 형태: {train_df.shape}")
print(f"🎯 타겟 분포: {train_df['rule_violation'].value_counts().to_dict()}")
print("\n📋 데이터 샘플:")
display(train_df.head())



🔍 데이터 형태: (2029, 9)
🎯 타겟 분포: {1: 1031, 0: 998}

📋 데이터 샘플:


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [4]:
# ==========================================================
# 🛠️ 특징 엔지니어링 함수 정의
def count_urls(text):
    """URL 개수 세기"""
    return len(re.findall(r'https?://\S+|www\.\S+', str(text)))

def count_exclaims(text):
    """감탄표 개수 세기"""
    return str(text).count('!')

def count_questions(text):
    """물음표 개수 세기"""
    return str(text).count('?')

def upper_ratio(text):
    """대문자 비율 계산"""
    s = str(text)
    letters = [c for c in s if c.isalpha()]
    if not letters:
        return 0.0
    upp = sum(1 for c in letters if c.isupper())
    return upp / len(letters)

def repeat_char_max(text):
    """연속된 문자의 최대 길이"""
    longest = 1
    last = ''
    cur = 0
    for ch in str(text):
        if ch == last:
            cur += 1
        else:
            longest = max(longest, cur)
            cur = 1
            last = ch
    longest = max(longest, cur)
    return longest

def jaccard_similarity(text1, text2):
    """자카드 유사도 계산"""
    set1 = set(str(text1).lower().split())
    set2 = set(str(text2).lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

# ==========================================================
# 🔧 특징 생성 함수
def create_features(df):
    """EDA에서 발견한 유용한 특징들을 생성"""
    df = df.copy()
    
    print("📏 기본 텍스트 특징 생성 중...")
    df['body_len'] = df['body'].astype(str).str.len()
    df['rule_len'] = df['rule'].astype(str).str.len()
    df['body_words'] = df['body'].astype(str).str.split().str.len()
    
    print("🎨 스타일 특징 생성 중...")
    df['url_cnt'] = df['body'].apply(count_urls)
    df['exc_cnt'] = df['body'].apply(count_exclaims)
    df['q_cnt'] = df['body'].apply(count_questions)
    df['upper_rt'] = df['body'].apply(upper_ratio)
    df['rep_run'] = df['body'].apply(repeat_char_max)
    
    print("😊 이모지 및 특수문자 특징 생성 중...")
    emoji_set = set("😀😃😄😁😆😅😂🤣🥲😊🙂🙃😉😍😘😗😙😚😋😛😝😜🤪🤨🧐🤓😎🤩🥳😏😒😞😔😟😕🙁☹️😣😖😫😩😤😠😡🤬")
    df['emoji_rt'] = df['body'].apply(
        lambda s: sum(1 for ch in str(s) if ch in emoji_set) / max(len(str(s)), 1)
    )
    
    print("📚 어휘 기반 특징 생성 중...")
    profanity_words = {'idiot', 'moron', 'stupid', 'dumb', 'retard', 'asshole', 
                      'bastard', 'fuck', 'shit', 'damn', 'bitch'}
    ad_words = {'free', 'win', 'offer', 'discount', 'promo', 'sale', 'subscribe',
                'click', 'visit', 'buy', 'deal', 'coupon', 'limited'}
    
    df['has_profanity'] = df['body'].apply(
        lambda x: int(any(word in str(x).lower() for word in profanity_words))
    )
    df['has_adword'] = df['body'].apply(
        lambda x: int(any(word in str(x).lower() for word in ad_words))
    )
    
    print("🔗 규칙-댓글 상호작용 특징 생성 중...")
    df['rule_body_jaccard'] = [
        jaccard_similarity(rule, body) 
        for rule, body in zip(df['rule'], df['body'])
    ]
    
    print("✅ 특징 생성 완료!")
    return df

# ==========================================================
# 🤖 Cross-Encoder 입력 준비 함수
def prepare_cross_encoder_input(rule, body, positive_ex1=None, positive_ex2=None, 
                               negative_ex1=None, negative_ex2=None):
    """Cross-Encoder를 위한 입력 텍스트 준비"""
    rule_text = str(rule).strip()
    comment_text = str(body).strip()
    
    examples_text = ""
    if pd.notna(positive_ex1) and str(positive_ex1).strip():
        examples_text += f" [긍정예시] {str(positive_ex1).strip()}"
    if pd.notna(negative_ex1) and str(negative_ex1).strip():
        examples_text += f" [부정예시] {str(negative_ex1).strip()}"
    
    full_input = f"[규칙] {rule_text}{examples_text} [댓글] {comment_text}"
    return full_input

# ==========================================================
# 📊 데이터 전처리 및 특징 생성
print("🔧 데이터 전처리 시작...")

# 특징 생성 실행
train_df = create_features(train_df)



🔧 데이터 전처리 시작...
📏 기본 텍스트 특징 생성 중...
🎨 스타일 특징 생성 중...
😊 이모지 및 특수문자 특징 생성 중...
📚 어휘 기반 특징 생성 중...
🔗 규칙-댓글 상호작용 특징 생성 중...
✅ 특징 생성 완료!


In [5]:
# 수치 특징 정의
numerical_cols = [
    'body_len', 'rule_len', 'body_words', 'url_cnt', 'exc_cnt', 'q_cnt',
    'upper_rt', 'rep_run', 'emoji_rt', 'has_profanity', 'has_adword', 
    'rule_body_jaccard'
]

# 수치 특징 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_features = scaler.fit_transform(train_df[numerical_cols])

# Cross-Encoder 입력 데이터 준비
print("🔄 Cross-Encoder 입력 데이터 준비 중...")
ce_inputs = []
labels = []

for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="입력 데이터 처리"):
    ce_input = prepare_cross_encoder_input(
        row['rule'], 
        row['body'],
        row.get('positive_example_1'),
        row.get('positive_example_2'),
        row.get('negative_example_1'),
        row.get('negative_example_2')
    )
    ce_inputs.append(ce_input)
    labels.append(int(row['rule_violation']))

ce_inputs = np.array(ce_inputs)
labels = np.array(labels)

print(f"✅ {len(ce_inputs)}개의 Cross-Encoder 입력 쌍 준비 완료")



🔄 Cross-Encoder 입력 데이터 준비 중...


입력 데이터 처리: 100%|██████████| 2029/2029 [00:00<00:00, 17049.91it/s]

✅ 2029개의 Cross-Encoder 입력 쌍 준비 완료


In [ ]:
# ==========================================================
# 🏗️ 최종 모델 훈련 (전체 데이터 사용)
from sentence_transformers import CrossEncoder, InputExample

print("🏗️ 전체 데이터셋으로 최종 모델 훈련 시작!")

# 최종 모델 초기화
model_name = 'microsoft/deberta-v3-small'
final_model = CrossEncoder(model_name, num_labels=1, device=device)

# 전체 훈련 데이터로 예시 생성
print("📚 전체 훈련 예시 생성 중...")
train_examples = []

for i in tqdm(range(len(ce_inputs)), desc="최종 훈련 데이터 처리"):
    ce_input = ce_inputs[i]
    
    if '[댓글]' in ce_input:
        rule_part = ce_input.split('[댓글]')[0].strip()
        comment_part = ce_input.split('[댓글]')[1].strip()
    else:
        parts = ce_input.split()
        mid = len(parts) // 2
        rule_part = ' '.join(parts[:mid])
        comment_part = ' '.join(parts[mid:])
    
    train_examples.append(
        InputExample(texts=[rule_part, comment_part], label=float(labels[i]))
    )

# 최종 훈련 설정
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
warmup_steps = max(1, int(len(train_dataloader) * 0.1))

print(f"🚀 최종 모델 훈련 (에폭: 4, 배치: 16)")

# 최종 모델 훈련
final_model.fit(
    train_dataloader=train_dataloader,
    epochs=4,
    warmup_steps=warmup_steps,
    output_path='/kaggle/working/final_cross_encoder_model',
    save_best_model=True,
    show_progress_bar=True
)



🏗️ 전체 데이터셋으로 최종 모델 훈련 시작!


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📚 전체 훈련 예시 생성 중...


최종 훈련 데이터 처리: 100%|██████████| 2029/2029 [00:00<00:00, 156042.44it/s]


🚀 최종 모델 훈련 (에폭: 4, 배치: 16)


Epoch:  25%|██▌       | 1/4 [03:44<11:12, 224.10s/it]

In [ ]:
# ==========================================================
# 💾 모델 및 전처리 객체 저장
print("💾 모델 및 전처리 객체 저장 중...")

# 1. Cross-Encoder 모델은 이미 output_path에 저장됨
print("✅ Cross-Encoder 모델 저장 완료: /kaggle/working/final_cross_encoder_model")

# 2. Scaler 저장
joblib.dump(scaler, '/kaggle/working/scaler.pkl')
print("✅ Scaler 저장 완료: /kaggle/working/scaler.pkl")

# 3. 수치 특징 컬럼명 저장
with open('/kaggle/working/numerical_cols.pkl', 'wb') as f:
    pickle.dump(numerical_cols, f)
print("✅ 수치 특징 컬럼명 저장 완료: /kaggle/working/numerical_cols.pkl")

# 4. 특징 엔지니어링 함수들을 포함한 유틸리티 저장
feature_utils = {
    'count_urls': count_urls,
    'count_exclaims': count_exclaims,
    'count_questions': count_questions,
    'upper_ratio': upper_ratio,
    'repeat_char_max': repeat_char_max,
    'jaccard_similarity': jaccard_similarity,
    'create_features': create_features,
    'prepare_cross_encoder_input': prepare_cross_encoder_input
}

with open('/kaggle/working/feature_utils.pkl', 'wb') as f:
    pickle.dump(feature_utils, f)
print("✅ 특징 엔지니어링 함수들 저장 완료: /kaggle/working/feature_utils.pkl")



In [ ]:
# ==========================================================
# 📋 저장된 파일들 확인
print("\n📋 저장된 파일들:")
for file in os.listdir('/kaggle/working'):
    if file.endswith(('.pkl', '.bin', '.json', '.txt')) or 'model' in file:
        file_path = f'/kaggle/working/{file}'
        file_size = os.path.getsize(file_path) / (1024*1024)  # MB
        print(f"  - {file}: {file_size:.2f} MB")

print(f"\n🎉 모델 학습 및 저장 완료!")
print(f"📦 이제 이 파일들을 Kaggle Dataset으로 업로드하여 추론 노트북에서 사용하세요.")
print(f"📁 업로드할 폴더: /kaggle/working/")

# 저장된 모델로 빠른 테스트
print("\n🧪 저장된 모델 빠른 테스트...")
test_rule = "No spam or promotional content"
test_body = "Check out this amazing deal! Buy now!"

test_input = prepare_cross_encoder_input(test_rule, test_body)
if '[댓글]' in test_input:
    rule_part = test_input.split('[댓글]')[0].strip()
    comment_part = test_input.split('[댓글]')[1].strip()
else:
    parts = test_input.split()
    mid = len(parts) // 2
    rule_part = ' '.join(parts[:mid])
    comment_part = ' '.join(parts[mid:])

test_pred = final_model.predict([[rule_part, comment_part]])
test_prob = torch.sigmoid(torch.tensor(test_pred)).item()

print(f"테스트 규칙: {test_rule}")
print(f"테스트 댓글: {test_body}")
print(f"위반 확률: {test_prob:.4f}")

print("\n✅ 모든 작업 완료!")